# D1: Stage 2 - Depuracion Imagenes No Originales
Subset general a no tener en cuenta el subset malo. Cambio

In [ ]:
#Subset with all the data (images)
image_dir = "ANPR2.v1i.yolov8/train/images"

#The data can be grouped in subsets of 3 images. The 3 images share the prefix befor .rf. One of them is the original file, the other 2 are the result of data augmentation
# Example: _129602646_img_4997_jpg.rf.8dee979ae148bf9df2d5ab9554c76c08.jpg, _129602646_img_4997_jpg.rf.adfe2fcc93083228344f72aff4d7eefa.jpg and _129602646_img_4997_jpg.rf.b7e35a0afa1b639ed174e166967bb157.jpg
# Original File on the Example: _129602646_img_4997_jpg.rf.adfe2fcc93083228344f72aff4d7eefa.jpg

#CSV with the names of the file of the images that are not the original ones (need to be removed)
#The first 3 lines of the csv files are:
#_129602646_img_4997_jpg.rf.8dee979ae148bf9df2d5ab9554c76c08
#1be42486b669cf3f7b8a4bbc89f2b776_jpg.rf.1eec3aedb0c0f224b2c95d9342d6fc30
#6O63HEQHE5E3PCNEABPOXWW7BQ_jpeg.rf.11b2eabf399c40a789930fa35a79028c
invalid_dir = "limpieza_dataset1_stage1.csv"
invalid_dir = "limpieza_dataset_placas_stage-stage2.csv"

#the prefixes which are not mentioned in limpieza_dataset1_stage1.csv nor  limpieza_dataset_placas_stage-stage2.csv" are the subsets that i know which ones are the orignal ones.
#(OLD) the prefixes mentioned on limpieza_dataset1_stage1.csv are the ones that i still need to know which ones are the orignal ones
#(NEW) the prefixes that are metnioned both on limpieza_dataset1_stage1.csv and limpieza_dataset_placas_stage-stage2.csv have only one option left that is the original one.

#create a python script to:
#1. create a data structure with all the file names on "ANPR2.v1i.yolov8/train/images"
#2. create a data structure with the prefixes which need to be reviewed in order to find the orignal ones
#3. (OLD) pick one of the two remain optione and save them on ANPR2.v1i.yolov8/train_stage2_options/images. crea the subfoldders if they dont exist
#3. (NEW) pick the only option left and savethem them on ANPR2.v1i.yolov8/train_stage3_options/images. crea the subfoldders if they dont exist

In [ ]:
import os
import shutil
import pandas as pd
from collections import defaultdict

# Define paths
image_dir = "ANPR2.v1i.yolov8/train/images"
invalid_file = "limpieza_dataset1_stage1.csv"
output_dir = "ANPR2.v1i.yolov8/train_stage2_options/images"

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Step 1: Read all image filenames
image_files = sorted(os.listdir(image_dir))

# Step 2: Load invalid prefixes (from CSV)
invalid_prefixes = set()
if os.path.exists(invalid_file):
    df_invalid = pd.read_csv(invalid_file, header=None)
    for filename in df_invalid[0]:
        prefix = filename.split(".rf.")[0]  # Extract prefix
        invalid_prefixes.add(prefix)

# Step 3: Group images by prefix
grouped_images = defaultdict(list)
for image in image_files:
    if ".rf." in image:
        prefix = image.split(".rf.")[0]  # Extract the shared prefix
        grouped_images[prefix].append(image)

# Step 4: Select one of the remaining options for review
for prefix, files in grouped_images.items():
    if prefix in invalid_prefixes:
        remaining_files = [f for f in files if f.split(".rf.")[0] == prefix]
        
        # Select the last one in sorted order (arbitrary choice)
        if remaining_files:
            selected_file = remaining_files[-1]
            
            # Copy selected image to train_stage2_options/images
            src_path = os.path.join(image_dir, selected_file)
            dest_path = os.path.join(output_dir, selected_file)
            shutil.copy2(src_path, dest_path)

print("Images for stage 2 review copied successfully.")


Total images found: 2397
Invalid files from stage 1: 510
Invalid files from stage 2: 256
Common invalid prefixes (intersection): 0
Total prefixes grouped: 797

Images copied for stage 3 review: 0
Process completed.


# D1 Stage 3 - Continuando Depuracion

In [8]:
import os
import shutil
import pandas as pd
from collections import defaultdict

# Define paths
image_dir = "ANPR2.v1i.yolov8/train/images"
invalid_file1 = "limpieza_dataset1_stage1.csv"
invalid_file2 = "limpieza_dataset_placas_stage-stage2.csv"
output_dir = "ANPR2.v1i.yolov8/train_stage3_options/images"

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Step 1: Read all image filenames
image_files = sorted(os.listdir(image_dir))
print(f"Total images found: {len(image_files)}")

# Step 2: Load invalid prefixes from both CSVs (use prefixes, not full filenames)
invalid_prefixes_stage1 = set()
if os.path.exists(invalid_file1):
    df_invalid1 = pd.read_csv(invalid_file1, header=None)
    for filename in df_invalid1[0]:
        prefix = filename.split(".rf.")[0]  # Extract prefix
        invalid_prefixes_stage1.add(prefix)
    print(f"Invalid prefixes from stage 1: {len(invalid_prefixes_stage1)}")

invalid_prefixes_stage2 = set()
if os.path.exists(invalid_file2):
    df_invalid2 = pd.read_csv(invalid_file2, header=None)
    for filename in df_invalid2[0]:
        prefix = filename.split(".rf.")[0]  # Extract prefix
        invalid_prefixes_stage2.add(prefix)
    print(f"Invalid prefixes from stage 2: {len(invalid_prefixes_stage2)}")

# Find prefixes mentioned in both invalid lists
common_invalid_prefixes = invalid_prefixes_stage1.intersection(invalid_prefixes_stage2)
print(f"Common invalid prefixes (intersection): {len(common_invalid_prefixes)}")

# Step 3: Group images by prefix
grouped_images = defaultdict(list)
for image in image_files:
    if ".rf." in image:
        prefix = image.split(".rf.")[0]  # Extract the shared prefix
        grouped_images[prefix].append(image)

print(f"Total prefixes grouped: {len(grouped_images)}")

# Step 4: Select the only remaining option for each prefix
copied_count = 0
for prefix in common_invalid_prefixes:
    if prefix in grouped_images:
        files = grouped_images[prefix]
        print(f"\nChecking prefix: {prefix}")
        print(f"Files under prefix {prefix}: {files}")

        # Filter out images that are listed as invalid (by filename)
        invalid_fullnames = set(df_invalid1[0].tolist() + df_invalid2[0].tolist())
        remaining_files = [f for f in files if f.replace('.jpg', '') not in invalid_fullnames]

        print(f"Remaining files for prefix {prefix}: {remaining_files}")

        # If only one option remains, copy it to output directory
        if len(remaining_files) == 1:
            selected_file = remaining_files[0]
            print(f"✅ Selected file for prefix {prefix}: {selected_file}")

            src_path = os.path.join(image_dir, selected_file)
            dest_path = os.path.join(output_dir, selected_file)
            shutil.copy2(src_path, dest_path)
            copied_count += 1
        else:
            print(f"⚠️ Multiple or no options found for prefix {prefix}, skipping...")
    else:
        print(f"❌ No files grouped for prefix {prefix}, skipping...")

print(f"\n✅ Images copied for stage 3 review: {copied_count}")
print("✅ Process completed.")


Total images found: 2397
Invalid prefixes from stage 1: 510
Invalid prefixes from stage 2: 256
Common invalid prefixes (intersection): 256
Total prefixes grouped: 797

Checking prefix: 20231009_195421_jpg
Files under prefix 20231009_195421_jpg: ['20231009_195421_jpg.rf.24c79ab59094e0e5636be066a8d42242.jpg', '20231009_195421_jpg.rf.2bc27be0d1f61d4f4ae95c21aefd3b77.jpg', '20231009_195421_jpg.rf.df21c6aea266469f179cf95332a4cf24.jpg']
Remaining files for prefix 20231009_195421_jpg: ['20231009_195421_jpg.rf.2bc27be0d1f61d4f4ae95c21aefd3b77.jpg']
✅ Selected file for prefix 20231009_195421_jpg: 20231009_195421_jpg.rf.2bc27be0d1f61d4f4ae95c21aefd3b77.jpg

Checking prefix: Foto-Placa-84-_jpg
Files under prefix Foto-Placa-84-_jpg: ['Foto-Placa-84-_jpg.rf.a34f09103c10924c4b43dd8026cefca5.jpg', 'Foto-Placa-84-_jpg.rf.f254044e5dd4d2b76d911b85ede2d73a.jpg', 'Foto-Placa-84-_jpg.rf.f5f921f6d764af753fb81d787b7b1062.jpg']
Remaining files for prefix Foto-Placa-84-_jpg: ['Foto-Placa-84-_jpg.rf.f254044e5dd

# D1 Stage 4 - Union

now we have the following facts
1. its possible to obtain the list from ALL the files. (ANPR2.v1i.yolov8\train\images)
2. Images in (ANPR2.v1i.yolov8\train_filter_v2\images) which are not mentioned in limpieza_dataset1_stage1.csv are the ORIGINAL ONES from that subset.
3. Images in (ANPR2.v1i.yolov8\train_stage2_options\images) which are not mentioned in limpieza_dataset_placas_stage-stage2.csv are the ORIGINAL ONES from that subset.
3. Images in (ANPR2.v1i.yolov8\train_stage3_options\images) should be the correct ones that are missing.

From, that make a datastructure that stores all the ORIGNAL images and saves them on a .csv.

In [10]:
import os
import pandas as pd

# Define paths
filter_v2_dir = "ANPR2.v1i.yolov8/train_filter_v2/images"
stage2_dir = "ANPR2.v1i.yolov8/train_stage2_options/images"
stage3_dir = "ANPR2.v1i.yolov8/train_stage3_options/images"

filter_v2_invalid_file = "limpieza_dataset1_stage1.csv"
stage2_invalid_file = "limpieza_dataset_placas_stage-stage2.csv"
output_csv = "original_images.csv"

# Step 1: Helper function to extract prefix (before .rf.)
def get_prefix(filename):
    return filename.split(".rf.")[0]

# Step 2: Load invalid prefixes from CSV files (prefixes only)
filter_v2_invalid_prefixes = set(pd.read_csv(filter_v2_invalid_file, header=None)[0].apply(get_prefix))
print(f"Invalid prefixes from filter_v2: {len(filter_v2_invalid_prefixes)}")

stage2_invalid_prefixes = set(pd.read_csv(stage2_invalid_file, header=None)[0].apply(get_prefix))
print(f"Invalid prefixes from stage 2: {len(stage2_invalid_prefixes)}")

# Step 3: Get original images from Filter V2
filter_v2_images = set(os.listdir(filter_v2_dir))
filter_v2_original = {img for img in filter_v2_images if get_prefix(img) not in filter_v2_invalid_prefixes}
print(f"Original images from filter_v2: {len(filter_v2_original)}")

# Step 4: Get original images from Stage 2
stage2_images = set(os.listdir(stage2_dir))
stage2_original = {img for img in stage2_images if get_prefix(img) not in stage2_invalid_prefixes}
print(f"Original images from stage 2: {len(stage2_original)}")

# Step 5: Get original images from Stage 3 (remaining after subsets 1 and 2)
stage3_images = set(os.listdir(stage3_dir))
already_accounted_prefixes = {get_prefix(img) for img in filter_v2_original.union(stage2_original)}
stage3_original = {img for img in stage3_images if get_prefix(img) not in already_accounted_prefixes}
print(f"Original images from stage 3: {len(stage3_original)}")

# Step 6: Combine all original images
all_original_images = sorted(filter_v2_original.union(stage2_original, stage3_original))
print(f"Total original images collected: {len(all_original_images)}")

# Step 7: Save to CSV
df_original = pd.DataFrame(all_original_images, columns=["original_images"])
df_original.to_csv(output_csv, index=False)
print(f"✅ Original images saved to {output_csv}")


Invalid prefixes from filter_v2: 510
Invalid prefixes from stage 2: 256
Original images from filter_v2: 287
Original images from stage 2: 254
Original images from stage 3: 255
Total original images collected: 796
✅ Original images saved to original_images.csv


In [12]:
#copy paste
import os
import shutil
import pandas as pd

# Step 1: Define paths
original_images_file = "original_images.csv"
train_images_dir = "ANPR2.v1i.yolov8/train/images"
output_dir = "ANPR2.v1i.yolov8/train_stage4_options/images"

# Step 2: Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Step 3: Load selected original image filenames from CSV
df = pd.read_csv(original_images_file)
original_images = set(df["original_images"].tolist())
print(f"Total selected original images: {len(original_images)}")

# Step 4: Copy matching images from train/images to train_stage4_options/images
copied_count = 0
for image in original_images:
    src_path = os.path.join(train_images_dir, image)
    dest_path = os.path.join(output_dir, image)

    if os.path.exists(src_path):
        shutil.copy2(src_path, dest_path)
        copied_count += 1
    else:
        print(f"❌ Image not found: {image}")

print(f"✅ Total images copied: {copied_count}")
print(f"Images saved to: {output_dir}")


Total selected original images: 796
❌ Image not found: 1-Municipalidad-Provincial-de-Arequipa-reiniciaria-tramite-de-entrega-de-brevetes-para-conductores-de-motos-en-abril-de-este-ano__jpg.rf.1acac56fc232f324eaa6097f86cfb21c.jpg
✅ Total images copied: 795
Images saved to: ANPR2.v1i.yolov8/train_stage4_options/images


# Contar nro. imagenes

In [9]:
#How many files
import os

# Set the directory path
output_dir = "Peru Plate Numbers.v3i.yolov8/train/images"
#output_dir = "ANPR2.v1i.yolov8/train_stage4_options/images"

# Define image file extensions
image_extensions = {".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff"}

# List all files in the directory
all_files = os.listdir(output_dir)

# Filter files by extension (case-insensitive)
image_files = [f for f in all_files if os.path.splitext(f)[1].lower() in image_extensions]

# Print the number of image files
print("Number of image files:", len(image_files))


Number of image files: 1470


## Comparacion de prefixes, original

In [17]:
import os

# Define directories
stage4_dir = "Peru Plate Numbers.v3i.yolov8/train_filter_v6/images"
#stage4_dir = "ANPR2.v1i.yolov8/train_stage6_options/images"
#train_dir = "ANPR2.v1i.yolov8/train/images"
train_dir = "Peru Plate Numbers.v3i.yolov8/train/images"

def get_unique_prefixes(directory):
    prefixes = set()
    for filename in os.listdir(directory):
        if '.rf' in filename:
            prefix = filename.split('.rf')[0]
            prefixes.add(prefix)
    return prefixes

prefixes_stage4 = get_unique_prefixes(stage4_dir)
prefixes_train = get_unique_prefixes(train_dir)

missing_prefixes = prefixes_train - prefixes_stage4

#print("Unique prefixes in stage4 images directory:")
#print(prefixes_stage4)
#print("\nUnique prefixes in train images directory:")
#print(prefixes_train)
print("Count of unique prefixes in stage6_dir images directory:", len(prefixes_stage4))
print("Count of unique prefixes in train images directory:", len(prefixes_train))
#print("\nPrefixes missing in stage4 images directory:")
#for prefix in sorted(missing_prefixes):
#    print(prefix)


Count of unique prefixes in stage6_dir images directory: 450
Count of unique prefixes in train images directory: 490


### Version 2

In [4]:
import os
from PIL import Image

# Define directories
stage4_dir = "ANPR2.v1i.yolov8/train_stage5_options/images"
#train_dir = "ANPR2.v1i.yolov8/train/images"
train_dir = "ANPR2.v1i.yolov8/train_del/train_filter_v2/images"

def get_unique_prefixes(directory):
    prefixes = set()
    for filename in os.listdir(directory):
        if '.rf' in filename:
            prefix = filename.split('.rf')[0]
            prefixes.add(prefix)
    return prefixes

def get_unique_extensions(directory):
    extensions = set()
    for filename in os.listdir(directory):
        ext = os.path.splitext(filename)[1].lower()
        extensions.add(ext)
    return extensions

def get_unique_dimensions(directory):
    dimensions = set()
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            with Image.open(file_path) as img:
                dimensions.add(img.size)
        except Exception:
            continue
    return dimensions

# Get unique prefixes
prefixes_stage4 = get_unique_prefixes(stage4_dir)
prefixes_train = get_unique_prefixes(train_dir)
missing_prefixes = prefixes_train - prefixes_stage4

print("Count of unique prefixes in stage images directory:", len(prefixes_stage4))
print("Count of unique prefixes in mod images directory:", len(prefixes_train))
print("\nPrefixes missing in stage4 images directory:")
for prefix in sorted(missing_prefixes):
    print(prefix)

# Get unique extensions and dimensions for stage4 directory
extensions_stage4 = get_unique_extensions(stage4_dir)
dimensions_stage4 = get_unique_dimensions(stage4_dir)

print("\nUnique extensions in stage4 images directory:")
for ext in sorted(extensions_stage4):
    print(ext)
print("\nUnique dimensions in stage4 images directory:")
for dim in sorted(dimensions_stage4):
    print(dim)

# Get unique extensions and dimensions for train directory
extensions_train = get_unique_extensions(train_dir)
dimensions_train = get_unique_dimensions(train_dir)

print("\nUnique extensions in train images directory:")
for ext in sorted(extensions_train):
    print(ext)
print("\nUnique dimensions in train images directory:")
for dim in sorted(dimensions_train):
    print(dim)


Count of unique prefixes in stage images directory: 795
Count of unique prefixes in mod images directory: 437

Prefixes missing in stage4 images directory:

Unique extensions in stage4 images directory:
.jpg

Unique dimensions in stage4 images directory:
(640, 640)

Unique extensions in train images directory:
.jpg

Unique dimensions in train images directory:
(640, 640)


# D1 1 Eliminacion de imagnes no relevantes
Motos y placas no visibles

In [6]:
import os
import csv

# Directories for the images
stage6_dir = "ANPR2.v1i.yolov8/train_stage6_options/images"
filter_dir = "ANPR2.v1i.yolov8/train_del/train_filter_v2/images"

def get_prefixes(directory):
    """
    Returns a dictionary mapping each unique prefix (the part before ".rf")
    to a list of corresponding filenames in the given directory.
    """
    prefix_to_files = {}
    for filename in os.listdir(directory):
        if ".rf" in filename:
            prefix = filename.split(".rf")[0]
            prefix_to_files.setdefault(prefix, []).append(filename)
    return prefix_to_files

# Get prefix mappings from both directories
stage6_prefixes = get_prefixes(stage6_dir)
filter_prefixes = get_prefixes(filter_dir)

# Determine unique prefixes in stage6 that are not in filter_dir
unique_stage6_prefixes = set(stage6_prefixes.keys()) - set(filter_prefixes.keys())

# Delete files from stage6 with the unique prefixes
for prefix in unique_stage6_prefixes:
    for filename in stage6_prefixes[prefix]:
        file_path = os.path.join(stage6_dir, filename)
        os.remove(file_path)

# Save the deleted prefixes to a CSV file
csv_file = "deleted_prefixes.csv"
with open(csv_file, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["prefix"])
    for prefix in unique_stage6_prefixes:
        writer.writerow([prefix])

# Print the number of unique prefixes that were deleted
print(f"Deleted {len(unique_stage6_prefixes)} unique prefixes. CSV saved to {csv_file}.")


Deleted 358 unique prefixes. CSV saved to deleted_prefixes.csv.


# D2 Stage 3

in "Peru Plate Numbers.v3i.yolov8/train/images"
i have images that are named like

20231009_192435.rf.77c0fc7846580623d2466871fadb5056
20231009_192435.rf.102ad47e3f2d2a9dad4dc37c33e5e30a
20231009_192435.rf.14919a38e97c954674d53b1ecd39bb13

you can call the part before ".rf" the prefix of each image.

in "Peru Plate Numbers.v3i.yolov8/train_filter_v2/images" i have images with the same convention.

In "Peru Plate Numbers.v3i.yolov8/train/images", there are 3 images per prefix. In "Peru Plate Numbers.v3i.yolov8/train_filter_v2/images" there is one image per prefix.

I am trying to discover the original image per prefix. I picked random iamges per prefix and it resulted in "Peru Plate Numbers.v3i.yolov8/train_filter_v2/images".

In limpieza_dataset_placas_stage-dataset2.csv, i have a list of images which prefix still need to be reviewed. ie. images which prefix i need to pick the other option of the remaining 2.

Your job is to create a python script that 

1. stores the image names from "Peru Plate Numbers.v3i.yolov8/train/images"
2. stores the image names from "Peru Plate Numbers.v3i.yolov8/train_filter_v2/images"
3. stores the image names from limpieza_dataset_placas_stage-dataset2.csv (CAUTION: the names on the csv do not contain extension at the last part, for example 20231009_192443_jpg.rf.da5d595c40e613d9878567c3c34f2c1d)
4. stores prefixes that still need to be reviewed and saves them on dataset2_stage2_remaining.csv
5. for those prefixes, select one of the remaining 2 options from "Peru Plate Numbers.v3i.yolov8/train/images" and saves all of them in "Peru Plate Numbers.v3i.yolov8/train_filter_v3/images" , create the folders if necessary



In [13]:
import os
import csv
import shutil

# Directories and CSV files
train_dir = "Peru Plate Numbers.v3i.yolov8/train/images"
filter_v2_dir = "Peru Plate Numbers.v3i.yolov8/train_filter_v2/images"
filter_v3_dir = "Peru Plate Numbers.v3i.yolov8/train_filter_v3/images"
csv_input = "limpieza_dataset_placas_stage-dataset2.csv"
csv_output = "dataset2_stage2_remaining.csv"

def get_prefix(filename):
    # Extract the part before ".rf" (keeps the _jpg if present)
    return filename.split(".rf")[0]

def build_prefix_mapping(directory):
    mapping = {}
    for f in os.listdir(directory):
        if ".rf" in f:
            prefix = get_prefix(f)
            mapping.setdefault(prefix, []).append(f)
    return mapping

# Build mappings for train and filter_v2 directories.
train_mapping = build_prefix_mapping(train_dir)
filter_v2_mapping = build_prefix_mapping(filter_v2_dir)

# Process CSV file and build set of prefixes to review.
review_prefixes = set()
with open(csv_input, newline="") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        if row:
            entry = row[0].strip()
            prefix_csv = entry.split(".rf")[0]
            review_prefixes.add(prefix_csv)

# Save the review prefixes to a CSV file.
with open(csv_output, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["prefix"])
    for prefix in review_prefixes:
        writer.writerow([prefix])
print(f"Saved {len(review_prefixes)} prefixes to {csv_output}")

# Ensure destination folder exists.
if not os.path.exists(filter_v3_dir):
    os.makedirs(filter_v3_dir)

# For each prefix needing review, select one remaining image from train (excluding the filter_v2 choice)
selected_count = 0
for prefix in review_prefixes:
    train_images = train_mapping.get(prefix, [])
    if not train_images:
        print(f"No train images found for prefix: {prefix}")
        continue
    # In filter_v2 there should be one image per prefix.
    v2_images = filter_v2_mapping.get(prefix, [])
    chosen_v2 = v2_images[0] if v2_images else None
    # Exclude the filter_v2 image from train images.
    remaining = [img for img in train_images if img != chosen_v2]
    if not remaining:
        print(f"No remaining images for prefix: {prefix}")
        continue
    remaining.sort()
    selected_img = remaining[0]
    src_path = os.path.join(train_dir, selected_img)
    dst_path = os.path.join(filter_v3_dir, selected_img)
    shutil.copy(src_path, dst_path)
    selected_count += 1

print(f"Copied {selected_count} images to {filter_v3_dir}")


Saved 118 prefixes to dataset2_stage2_remaining.csv
Copied 118 images to Peru Plate Numbers.v3i.yolov8/train_filter_v3/images


# D2 Stage 4

in "Peru Plate Numbers.v3i.yolov8/train/images"
i have images that are named like

20231009_192435.rf.77c0fc7846580623d2466871fadb5056
20231009_192435.rf.102ad47e3f2d2a9dad4dc37c33e5e30a
20231009_192435.rf.14919a38e97c954674d53b1ecd39bb13

you can call the part before ".rf" the prefix of each image.

in "Peru Plate Numbers.v3i.yolov8/train_filter_v2/images" i have images with the same convention.

In "Peru Plate Numbers.v3i.yolov8/train/images", there are 3 images per prefix. In "Peru Plate Numbers.v3i.yolov8/train_filter_v2/images" there is one image per prefix.

I am trying to discover the original image per prefix. I picked random iamges per prefix and it resulted in "Peru Plate Numbers.v3i.yolov8/train_filter_v2/images".

In limpieza_dataset_placas_stage-dataset2.csv, i have a list of images which prefix still need to be reviewed.

I created "Peru Plate Numbers.v3i.yolov8/train_filter_v3/images" with the prefixes that still needed to be reviewed and picked 1 of the remaining 2 options left.

in "Peru Plate Numbers.v3i.yolov8/train_filter_v3/images" i have images with the same convention.

In limpieza_dataset_placas_stage-d2stage3.csv are the list of images which were not the original ones.

your job is to create a python script that

1. stores the image names  in "Peru Plate Numbers.v3i.yolov8/train/images"
2. stores the unique prefixes that still in limpieza_dataset_placas_stage-d2stage3.csv
3. creates "Peru Plate Numbers.v3i.yolov8/train_filter_v4/images" with the remaining option for original image per prefix that still need to be reviwed.


remember

example lines in limpieza_dataset_placas_stage-dataset2.csv

20231009_192443_jpg.rf.da5d595c40e613d9878567c3c34f2c1d
20231009_193512_jpg.rf.8e9857635ffeeb0f41dc785e634b365e
20231009_194138_jpg.rf.f8503a9a3ff10d681fdfd1b6f0221db3
20231009_202003_jpg.rf.edacee1547129faa230a30802214179b

example file names in Peru Plate Numbers.v3i.yolov8\train\images

20231009_192435.rf.77c0fc7846580623d2466871fadb5056.jpg
20231009_192435.rf.102ad47e3f2d2a9dad4dc37c33e5e30a.jpg
20231009_192435.rf.14919a38e97c954674d53b1ecd39bb13.jpg

example file names in Peru Plate Numbers.v3i.yolov8\train\images

20231009_192435.rf.77c0fc7846580623d2466871fadb5056.jpg
20231009_192435_jpg.rf.ee9a1439af23e23dfdd6054fa69f6d25.jpg
20231009_192443_jpg.rf.da5d595c40e613d9878567c3c34f2c1d.jpg
20231009_192448_jpg.rf.7116d5e98afd348ef687e046a5a78db7.jpg


In [14]:
import os
import csv
import shutil

# Define directories and CSV file
train_dir = "Peru Plate Numbers.v3i.yolov8/train/images"
filter_v2_dir = "Peru Plate Numbers.v3i.yolov8/train_filter_v2/images"
filter_v3_dir = "Peru Plate Numbers.v3i.yolov8/train_filter_v3/images"
filter_v4_dir = "Peru Plate Numbers.v3i.yolov8/train_filter_v4/images"
csv_d2stage3 = "limpieza_dataset_placas_stage-d2stage3.csv"

def get_prefix(filename):
    # Returns the part before ".rf"
    return filename.split(".rf")[0]

def build_prefix_mapping(directory):
    """
    Builds a dictionary mapping each prefix (extracted from filenames) to a list of filenames.
    """
    mapping = {}
    for f in os.listdir(directory):
        if ".rf" in f:
            prefix = get_prefix(f)
            mapping.setdefault(prefix, []).append(f)
    return mapping

# 1. Build mapping for train images (prefix -> list of filenames)
train_mapping = build_prefix_mapping(train_dir)

# Build mapping for filter_v2 images (one image per prefix)
v2_mapping = {}
for f in os.listdir(filter_v2_dir):
    if ".rf" in f:
        prefix = get_prefix(f)
        v2_mapping[prefix] = f

# Build mapping for filter_v3 images (one image per prefix)
v3_mapping = {}
for f in os.listdir(filter_v3_dir):
    if ".rf" in f:
        prefix = get_prefix(f)
        v3_mapping[prefix] = f

# 2. Read CSV file and extract unique prefixes to review
review_prefixes = set()
with open(csv_d2stage3, newline="") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        if row:
            entry = row[0].strip()
            prefix = entry.split(".rf")[0]
            review_prefixes.add(prefix)
print(f"Found {len(review_prefixes)} prefixes to review from CSV.")

# 3. For each prefix, determine the remaining candidate image and copy it to filter_v4
if not os.path.exists(filter_v4_dir):
    os.makedirs(filter_v4_dir)

copied_count = 0
for prefix in review_prefixes:
    # Get the list of train images for this prefix
    imgs = train_mapping.get(prefix, [])
    if not imgs:
        print(f"No train images found for prefix: {prefix}")
        continue
    # Exclude the image from filter_v2 (the one randomly picked)
    if prefix in v2_mapping:
        imgs = [img for img in imgs if img != v2_mapping[prefix]]
    # Exclude the image from filter_v3 (the non-original candidate)
    if prefix in v3_mapping:
        imgs = [img for img in imgs if img != v3_mapping[prefix]]
    # The remaining image should be the original candidate
    if len(imgs) == 1:
        src = os.path.join(train_dir, imgs[0])
        dst = os.path.join(filter_v4_dir, imgs[0])
        shutil.copy(src, dst)
        copied_count += 1
    else:
        print(f"Unexpected number of remaining images for prefix '{prefix}': {imgs}")

print(f"Copied {copied_count} images to {filter_v4_dir}")


Found 36 prefixes to review from CSV.
Copied 36 images to Peru Plate Numbers.v3i.yolov8/train_filter_v4/images


# D3 Stage 5 Unite

ok thanks, it works. Now i want to unite all the correct original images into "Peru Plate Numbers.v3i.yolov8/train_filter_v5/images (create the necessary folders)

there are 3 correct subsets

1. Images in "Peru Plate Numbers.v3i.yolov8/train_filter_v2/images" that do not appear in "limpieza_dataset_placas_stage-dataset2.csv"
2. Images in "Peru Plate Numbers.v3i.yolov8/train_filter_v3/images" that do not appear in "limpieza_dataset_placas_stage-d2stage3.csv"
3. Images in "Peru Plate Numbers.v3i.yolov8/train_filter_v4/images"

remember

example lines in limpieza_dataset_placas_stage-dataset2.csv and other csvs

20231009_192443_jpg.rf.da5d595c40e613d9878567c3c34f2c1d
20231009_193512_jpg.rf.8e9857635ffeeb0f41dc785e634b365e
20231009_194138_jpg.rf.f8503a9a3ff10d681fdfd1b6f0221db3
20231009_202003_jpg.rf.edacee1547129faa230a30802214179b

example file names in Peru Plate Numbers.v3i.yolov8\train\images

20231009_192435.rf.77c0fc7846580623d2466871fadb5056.jpg
20231009_192435.rf.102ad47e3f2d2a9dad4dc37c33e5e30a.jpg
20231009_192435.rf.14919a38e97c954674d53b1ecd39bb13.jpg

example file names in Peru Plate Numbers.v3i.yolov8\train\images

20231009_192435.rf.77c0fc7846580623d2466871fadb5056.jpg
20231009_192435_jpg.rf.ee9a1439af23e23dfdd6054fa69f6d25.jpg
20231009_192443_jpg.rf.da5d595c40e613d9878567c3c34f2c1d.jpg
20231009_192448_jpg.rf.7116d5e98afd348ef687e046a5a78db7.jpg


In [15]:
import os
import csv
import shutil

# Define directories
filter_v2_dir = "Peru Plate Numbers.v3i.yolov8/train_filter_v2/images"
filter_v3_dir = "Peru Plate Numbers.v3i.yolov8/train_filter_v3/images"
filter_v4_dir = "Peru Plate Numbers.v3i.yolov8/train_filter_v4/images"
filter_v5_dir = "Peru Plate Numbers.v3i.yolov8/train_filter_v5/images"

# CSV files
csv_v2 = "limpieza_dataset_placas_stage-dataset2.csv"
csv_v3 = "limpieza_dataset_placas_stage-d2stage3.csv"

def load_csv_set(csv_file):
    """Reads the CSV file and returns a set of image identifiers (filenames without extension)."""
    s = set()
    with open(csv_file, newline="") as f:
        reader = csv.reader(f)
        for row in reader:
            if row:
                # Each row is assumed to have one entry, e.g.
                # 20231009_192443_jpg.rf.da5d595c40e613d9878567c3c34f2c1d
                s.add(row[0].strip())
    return s

# Load CSV identifiers
csv_v2_set = load_csv_set(csv_v2)
csv_v3_set = load_csv_set(csv_v3)

# Create destination folder if it doesn't exist
os.makedirs(filter_v5_dir, exist_ok=True)

copied_count = 0

# Subset 1: Images from filter_v2 not in csv_v2_set
for filename in os.listdir(filter_v2_dir):
    if filename.lower().endswith(".jpg"):
        identifier = os.path.splitext(filename)[0]
        if identifier not in csv_v2_set:
            src_path = os.path.join(filter_v2_dir, filename)
            dst_path = os.path.join(filter_v5_dir, filename)
            shutil.copy(src_path, dst_path)
            copied_count += 1

# Subset 2: Images from filter_v3 not in csv_v3_set
for filename in os.listdir(filter_v3_dir):
    if filename.lower().endswith(".jpg"):
        identifier = os.path.splitext(filename)[0]
        if identifier not in csv_v3_set:
            src_path = os.path.join(filter_v3_dir, filename)
            dst_path = os.path.join(filter_v5_dir, filename)
            shutil.copy(src_path, dst_path)
            copied_count += 1

# Subset 3: All images from filter_v4
for filename in os.listdir(filter_v4_dir):
    if filename.lower().endswith(".jpg"):
        src_path = os.path.join(filter_v4_dir, filename)
        dst_path = os.path.join(filter_v5_dir, filename)
        shutil.copy(src_path, dst_path)
        copied_count += 1

print(f"Copied {copied_count} images to {filter_v5_dir}")


Copied 490 images to Peru Plate Numbers.v3i.yolov8/train_filter_v5/images


# D3 Stage 6 Delete

ok thanks, it works. Now i want the following.

in "Peru Plate Numbers.v3i.yolov8\train_filter_v6\images"
i have images that are named like

20231009_192435.rf.77c0fc7846580623d2466871fadb5056

you can call the part before ".rf" the prefix of each image.

In "Peru Plate Numbers.v3i.yolov8\train_filter_v6\images", there are the proposed images. In "Peru Plate Numbers.v3i.yolov8\train_del\images"
there are other images.

I want 

1. store the prefixes which appear in "Peru Plate Numbers.v3i.yolov8\train_filter_v6\images" but are not in "Peru Plate Numbers.v3i.yolov8\train_del\images" 
2. save them on a csv called deleted_prefixes_dataset2.csv
3. delete the images with said prefixes from "Peru Plate Numbers.v3i.yolov8\train_filter_v6\images"


In [16]:
import os
import csv

# Define directories and CSV output file
filter_v6_dir = r"Peru Plate Numbers.v3i.yolov8\train_filter_v6\images"
del_dir = r"Peru Plate Numbers.v3i.yolov8\train_del\images"
csv_output = "deleted_prefixes_dataset2.csv"

def get_prefix(filename):
    # Return the part before ".rf"
    return filename.split(".rf")[0]

def collect_prefixes(directory):
    prefixes = set()
    for f in os.listdir(directory):
        if ".rf" in f:
            prefixes.add(get_prefix(f))
    return prefixes

# Get prefixes from both directories
v6_prefixes = collect_prefixes(filter_v6_dir)
del_prefixes = collect_prefixes(del_dir)

# Determine prefixes in v6 that are not in del
prefixes_to_delete = v6_prefixes - del_prefixes

# Save these prefixes to CSV
with open(csv_output, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["prefix"])
    for prefix in sorted(prefixes_to_delete):
        writer.writerow([prefix])
print(f"Saved {len(prefixes_to_delete)} prefixes to {csv_output}")

# Delete images in filter_v6 with the identified prefixes
deleted_count = 0
for filename in os.listdir(filter_v6_dir):
    if ".rf" in filename:
        prefix = get_prefix(filename)
        if prefix in prefixes_to_delete:
            os.remove(os.path.join(filter_v6_dir, filename))
            deleted_count += 1

print(f"Deleted {deleted_count} images from {filter_v6_dir}")


Saved 40 prefixes to deleted_prefixes_dataset2.csv
Deleted 40 images from Peru Plate Numbers.v3i.yolov8\train_filter_v6\images
